# Python project: Getting ETF data


### Challange:

* The idea is to build a code that get data from etf.com and show it in a decent way. However, at first let's focus on the 'getting' part

# Step 1: Installing the necessary libraries and tools

#### In this case, I'll be using google

In [2]:
!pip install selenium

     ---------------------------------------- 6.5/6.5 MB 2.9 MB/s eta 0:00:00
     -------------------------------------- 384.9/384.9 kB 3.0 MB/s eta 0:00:00
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
     ---------------------------------------- 58.3/58.3 kB ? eta 0:00:00


In [6]:
!pip install webdriver-manager

# Step 2: Importing libraries.

In [87]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Step 3: Open connection and start collecting data.

In [88]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

driver.get('https://www.etf.com/etfanalytics/etf-finder')

# Step 3.1: Finding elements on page
#### Expanding list to 100 per page

In [90]:
#waiting the page to load
time.sleep(5)

btn_100 = driver.find_element("xpath",
                               '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button')

btn_100.click()


# Step 3.2: Finding elements on page
#### Getting the total number of pages

In [91]:
pg_number = driver.find_element("xpath", '//*[@id="totalPages"]')

pg_number = pg_number.text.replace("of ", "")

pg_number = int(pg_number)

pg_number

32

# Step 3.3: Getting the data table

In [92]:
elem = driver.find_element("xpath", '//*[@id="finderTable"]')

html_t = elem.get_attribute('outerHTML')

table = pd.read_html(str(html_t))[0]

table

,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$370.35B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$304.93B
2,VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.54B
3,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$280.86B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$172.61B
...,...,...,...,...,...,...
95,BIV,Vanguard Intermediate-Term Bond ETF,"Fixed Income: U.S. - Broad Market, Broad-based...",Vanguard,0.04%,$14.14B
96,VXF,Vanguard Extended Market ETF,Equity: U.S. - Extended Market,Vanguard,0.06%,$13.83B
97,HYG,iShares iBoxx USD High Yield Corporate Bond ETF,"Fixed Income: U.S. - Corporate, Broad-based Hi...",Blackrock,0.48%,$13.76B
98,XLI,Industrial Select Sector SPDR Fund,Equity: U.S. Industrials,State Street Global Advisors,0.10%,$13.64B


In [94]:
page_table = []

elem = driver.find_element("xpath", '//*[@id="finderTable"]')

for pg in range(1, pg_number + 1):
    
    html_t = elem.get_attribute('outerHTML')
    
    table = pd.read_html(str(html_t))[0]
    
    page_table.append(table)
    
    btn_next_page = driver.find_element("xpath", '//*[@id="nextPage"]')
    
    btn_next_page.click()
    
    
etfs_table = pd.concat(page_table)

etfs_table

,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$370.35B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$304.93B
2,VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.54B
3,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$280.86B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$172.61B
...,...,...,...,...,...,...
22,OAIB,Teucrium AiLA Long-Short Base Metals Strategy ETF,Commodities: Industrial Metals,Teucrium,--,--
23,IBOT,VanEck Robotics ETF,Equity: U.S. Robotics & AI,VanEck,--,--
24,CVSE,Calvert US Select Equity ETF,Equity: U.S. - Large Cap,Morgan Stanley,0.29%,--
25,SROI,Calamos Antetokounmpo Global Sustainable Equit...,Equity: Global - Total Market,Calamos Family Partners,0.95%,--


# Step 3.4: Finding elements on page:
#### Going back to page 1

In [95]:
go_back_page = driver.find_element("xpath", '//*[@id="goToPage"]')

go_back_page.clear()
go_back_page.send_keys("1")
go_back_page.send_keys(u'\ue007')

# Step 3.5: Finding elements on page:
#### Getting performance data

In [101]:
btn_performance = driver.find_element("xpath", '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/ul/li[2]')

btn_performance.click()

# Looping for getting the data

table_pages = []

elemen = driver.find_element("xpath", '//*[@id="finderTable"]')

for page in range(1, pg_number + 1):
    
    html_t = elemen.get_attribute('outerHTML')
    
    table = pd.read_html(str(html_t))[0]
    
    table_pages.append(table)
    
    btn_next_page = driver.find_element("xpath", '//*[@id="nextPage"]')
    
    driver.execute_script("arguments[0].click();", btn_next_page)
    
    
performance_table = pd.concat(table_pages)

performance_table

,Ticker,Name,1 Month,3 Month,YTD,1 Year,3 Years,5 Years,10 Years,As Of Date
0,SPY,SPDR S&P 500 ETF Trust,--,--,--,--,--,--,--,04/05/23
1,IVV,iShares Core S&P 500 ETF,--,--,--,--,--,--,--,04/05/23
2,VOO,Vanguard S&P 500 ETF,--,--,--,--,--,--,--,04/05/23
3,VTI,Vanguard Total Stock Market ETF,--,--,--,--,--,--,--,04/05/23
4,QQQ,Invesco QQQ Trust,--,--,--,--,--,--,--,04/05/23
...,...,...,...,...,...,...,...,...,...,...
22,OAIB,Teucrium AiLA Long-Short Base Metals Strategy ETF,--,--,--,--,--,--,--,04/05/23
23,IBOT,VanEck Robotics ETF,--,--,--,--,--,--,--,04/05/23
24,CVSE,Calvert US Select Equity ETF,--,--,--,--,--,--,--,04/05/23
25,SROI,Calamos Antetokounmpo Global Sustainable Equit...,--,--,--,--,--,--,--,04/05/23


In [102]:
#Closing driver
driver.quit()

# Setp 4: Building the final table.

In [103]:
performance_table = performance_table.set_index("Ticker")
performance_table = performance_table[['1 Year', '3 Years', '5 Years']]
etfs_table = etfs_table.set_index("Ticker")

In [104]:
final_db = etfs_table.join(tabela_rentabilidade_etfs, how = 'inner')

final_db

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,3 Years,5 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$370.35B,--,--,--
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$304.93B,--,--,--
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.54B,--,--,--
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$280.86B,--,--,--
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$172.61B,--,--,--
...,...,...,...,...,...,...,...,...
OAIB,Teucrium AiLA Long-Short Base Metals Strategy ETF,Commodities: Industrial Metals,Teucrium,--,--,--,--,--
IBOT,VanEck Robotics ETF,Equity: U.S. Robotics & AI,VanEck,--,--,--,--,--
CVSE,Calvert US Select Equity ETF,Equity: U.S. - Large Cap,Morgan Stanley,0.29%,--,--,--,--
